# Temporal Convolutional Network

Rough implementation of the encoder-decoder temporal convolutional network found in this paper: https://arxiv.org/abs/1611.05267

Model is not an exact replica of the one described in the paper, and a flattening layer was added before the output layer. Hyperparameters are also slightly different regarding the filter in each convolutional layer.

Tested the model on sequential MNIST data, which refers to a flattened version of each MNIST image. 

Model obtained 98.82% training accuracy and 96.81% testing accuracy.

Training time was nearly 34 minutes on an Nvidia GTX 1060

In [31]:
import tensorflow as tf
import keras.backend as K
from keras import optimizers
from keras.layers import Convolution1D, Dense, MaxPool1D
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard


In [37]:
from keras.datasets import mnist
batch_size = 128
num_classes = 10
epochs = 10

img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Reshaping it so that it is (60000, 784, 1) instead of (60000, 28, 28, 1) since it's not channels first
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows * img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows * img_cols)
    input_shape = (1, img_rows * img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols, 1)
input_shape = (img_rows * img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


print('Input shape: %s \nX_train shape: %s \nY_train shape: %s' % (input_shape, x_train.shape, y_train.shape))


Input shape: (784, 1) 
X_train shape: (60000, 784, 1) 
Y_train shape: (60000, 10)


In [33]:

#This is the model itself, adapted from the paper and slightly modified.


## ENCODER STAGE
input_layer = Input(shape=(input_shape ))

conv_encoder1 = keras.layers.convolutional.Conv1D(filters=int(input_shape[0]),
                                                   kernel_size=2,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(input_layer)
pool_1 = keras.layers.convolutional.MaxPooling1D(pool_size=2, strides=2)(conv_encoder1)

conv_encoder2 = keras.layers.convolutional.Conv1D(filters=int(input_shape[0]/2),
                                                   kernel_size=2,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(pool_1)

pool_2 = keras.layers.convolutional.MaxPooling1D(pool_size=2, strides=2)(conv_encoder2)

encoder = Dense(int(input_shape[0]/4), activation='relu')(pool_2)


## DECODER STAGE
upsample_1 = keras.layers.convolutional.UpSampling1D(size=2)(encoder)

conv_decoder1 = keras.layers.convolutional.Conv1D(filters=int(input_shape[0]/2),
                                                   kernel_size=2,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(upsample_1)

upsample_2 = keras.layers.convolutional.UpSampling1D(size=2)(conv_decoder1)

conv_decoder2 = keras.layers.convolutional.Conv1D(filters=input_shape[0],
                                                   kernel_size=2,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(upsample_2)
#Flattening layer to match y_test's shape
flat = keras.layers.Flatten()(conv_decoder2)
output_layer = Dense(num_classes, activation='softmax')(flat)

TCN = Model(inputs=input_layer, outputs=output_layer)


In [34]:
TCN.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="model_seqMnist.h5",
                               verbose=0,
                               save_best_only=True)

In [21]:
TCN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784, 1)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 784, 784)          2352      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 392, 784)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 392, 392)          615048    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 196, 392)          0         
_________________________________________________________________
dense_5 (Dense)              (None, 196, 196)          77028     
_________________________________________________________________
up_sampling1d_5 (UpSampling1 (None, 392, 196)          0         
__________

In [22]:
TCN.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks = [checkpointer])


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 209s - loss: 0.2286 - acc: 0.9296 - val_loss: 0.1552 - val_acc: 0.9497
Epoch 2/10
60000/60000 [==============================] - 202s - loss: 0.1322 - acc: 0.9598 - val_loss: 0.1238 - val_acc: 0.9623
Epoch 3/10
60000/60000 [==============================] - 204s - loss: 0.1090 - acc: 0.9682 - val_loss: 0.1168 - val_acc: 0.9653
Epoch 4/10
60000/60000 [==============================] - 203s - loss: 0.0934 - acc: 0.9717 - val_loss: 0.1060 - val_acc: 0.9669
Epoch 5/10
60000/60000 [==============================] - 203s - loss: 0.0810 - acc: 0.9762 - val_loss: 0.1162 - val_acc: 0.9654
Epoch 6/10
60000/60000 [==============================] - 203s - loss: 0.0708 - acc: 0.9786 - val_loss: 0.1099 - val_acc: 0.9665
Epoch 7/10
60000/60000 [==============================] - 202s - loss: 0.0609 - acc: 0.9819 - val_loss: 0.1120 - val_acc: 0.9671
Epoch 8/10
60000/60000 [=======================

In [23]:

score = TCN.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.12048382413223153
Test accuracy: 0.9681
